<a href="https://colab.research.google.com/github/karinayanh/Phyton_Assignments/blob/main/Sensitivity_analysis_on_Product_Mix_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Review on PC Tech Product Mix Optimization

## The PC Tech Company: Background

**The PC Tech Company** assembles and then tests two models of computers: **Basic** and **XP**.  
- PC Tech Company wants to know how many of each model it should produce for the coming month to **maximize its net profit**, given constraints on labor hours and sales limits.
- **No inventory** is carried over from the previous month, and PC Tech does not want to hold any inventory after this month.
- **Available labor hours:** 10,000 hours for assembly and 3,000 hours for testing  
- **No more than** 600 Basics and 1,200 XPs can be sold this month  

---

## Input Data Summary

| Category                | Basic        | XP          | Notes                  |
|--------------------------|--------------|-------------|------------------------|
| **Labor hours required**     | 5 (assembly) + 1 (testing) | 6 (assembly) + 2 (testing) | per computer |
| **Labor cost per hour**      | \$11 (assembly) | \$15 (testing) | applies to both models |
| **Component cost**           | \$150         | \$225        | per computer |
| **Selling price**            | \$300         | \$450        | per computer |
| **Available labor hours**    | 10,000 (assembly) | 3,000 (testing) | total available |
| **Maximum sales**            | 600           | 1,200        | units |


---

## Mathematical Model

### Step 1: Identify the Decision Variables
Let:

**$x_1$**: Number of Basic computers produced

**$x_2$**: Number of XP computers produced

Both variables must be integers.


### Step 2: Figure out Objective Function

Each computer earns profit = selling price − component cost − labor costs.
- Profit per Basic = 300 - 150 - 5 * 11 - 1 * 15 = \$80  
- Profit per XP = 450 - 225 - 6 * 11 - 2* 15 = \$129  

$$
\text{Maximize } 80x_1 + 129x_2
$$

### Step 3: Impose the Constraints
| Constraint | Equation | Description |
|-------------|-----------------------|-------------|
| Assembly hours | `5x₁ + 6x₂ ≤ 10000` | Assembly time limit |
| Testing hours | `x₁ + 2x₂ ≤ 3000` | Testing time limit |
| Sales limit (Basic) | `x₁ ≤ 600` | Sales limit |
| Sales limit (XP) | `x₂ ≤ 1200` | Sales limit |
| Nonnegativity | `x₁, x₂ ≥ 0` | Cannot produce negative units |

---

In [ ]:
import pyomo.environ as pyo
import pandas as pd
import numpy as np

# 0. Read data from Excel
file_path = "/content/Product Mix Problem.xlsx"
df = pd.read_excel(file_path, sheet_name=0, header=None) # Read entire sheet

# Read the coefficients, and then convert them to floats, and store them as a NumPy array.
c = df.iloc[1, 2:4].astype(float).to_numpy()   # Objective coefficients  C2:D2 (row index 1, columns 2 and 3)
A = df.iloc[3:9, 2:4].astype(float).to_numpy() # Constraint matrix  C4:D9 (row indices 3..8, columns 2..3)
b = df.iloc[3:9, 5].astype(float).to_numpy()   # Right-Hand Side Vector  F4:F9 (row indices 3..8, column 5)

m, n = A.shape
print(f"(m = {m} constraints, n = {n} variables)\n")
print(A)
print(b)

(m = 6 constraints, n = 2 variables)

[[5. 6.]
 [1. 2.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]]
[10000.  3000.   600.  1200.     0.     0.]


In [ ]:
# Example of a single constraint written explicitly (e.g., 5*x1 + 6*x2 <= 10000)
print(f" {A[0, 0] }*x1+{A[0, 1]}*x2 <= {b[0]}")

# Using a loop:
# With a single for-loop, we can generate all constraints from the matrix A and RHS vector b.

 5.0*x1+6.0*x2 <= 10000.0



## Python Implementation (Using Pyomo + HiGHS)

The modeling process follows exactly the same steps as the Beverage Planning Optimization

Difference: the decision variables are integers here

- **`pyo.ConcreteModel()`**  

- <span style="color:red"> **`pyo.Var(domain=pyo.NonNegativeIntegers)`** (integer, ≥0)</span>  

- **`pyo.Objective(expr=..., sense=pyo.maximize)`**

- **`pyo.ConstraintList()`** <span style="color:red"> (use ConstraintList and add constraints one by one)</span>

- **`pyo.SolverFactory("appsi_highs")`**

- **`solver.solve(model)`**   

- **`pyo.value(model.x1)`**

In [ ]:
# 1. Create the model
model = pyo.ConcreteModel(name="PC_Tech_Product_Mix")

# Define decision variables: (integer, ≥0)
model.x1 = pyo.Var(domain=pyo.NonNegativeReals)  # number of Basic computers
model.x2 = pyo.Var(domain=pyo.NonNegativeReals)  # number of XP computers

# Set the objective function: Maximize total net profit
model.obj = pyo.Objective(expr = c[0] * model.x1 + c[1] * model.x2, sense = pyo.maximize)

# Add Constraints
model.cons = pyo.ConstraintList() # use ConstraintList and add rows one by one (Ax <= b)
for i in range(m):
    if i<4:
        model.cons.add(A[i, 0] * model.x1 + A[i, 1] * model.x2 <= b[i]) # Labor hours limits and Sales limit
    else:
        model.cons.add(A[i, 0] * model.x1 + A[i, 1] * model.x2 >= b[i]) # Nonnegativity


# 2. Solve using APPsi–HiGHS
solver = pyo.SolverFactory("appsi_highs")
result = solver.solve(model)

# 3. Display results
print("\033[1mOptimal Integer Solution\033[0m")
print(f"Solver Status      : {result.solver.status}")
print(f"Termination Reason : {result.solver.termination_condition}\n")

x1 = int(pyo.value(model.x1))
x2 = int(pyo.value(model.x2))
profit = pyo.value(model.obj)

print(f"\033[1mOptimal Production Plan:\033[0m")
print(f"  x1 (Basic computers) : {x1} units") # The company produces 560 Basic computers
print(f"  x2 (XP computers)    : {x2} units") # The company produces 1200 XP computers
print(f"Total Profit: ${profit:,.2f}") # The company earns net profit $199,600

Optimal Integer Solution
Solver Status      : ok
Termination Reason : optimal

Optimal Production Plan:
  x1 (Basic computers) : 560 units
  x2 (XP computers)    : 1200 units
Total Profit: $199,600.00


# Sensitivity analysis

In real applications, the solution to a single model is hardly ever the end of the analysis. It is always useful to perform a sensitivity analysis to see how (or if) the solution changes as one or more inputs vary

In previous results, we have already identified the optimal decision and corresponding profits

- **Optimal production plan for basic computer : 560 units**
- **Optimal production plan for XP computer : 1200 units**
- **Optimal total profits : 199,600.00 units**

## Resource usage

Before changing any inputs, it's helpful to “audit” the optimal plan against the constraints. We plug the optimal production quantities back into each constraint to compute:

- **Total usage of each limited resource (e.g., assembly and testing hours)**

Constraints with zero slack are **binding**—they are the current bottlenecks that actively restrict the optimum. Constraints with positive slack are non-binding; increasing those limits (a little) will not change the optimal solution unless another constraint becomes binding.



In [ ]:
# Calculate total resource use
assembly_used = A[0,0]*x1 + A[0,1]*x2 # The company use 10000 assembly hours
testing_used  = A[1,0]*x1 + A[1,1]*x2 # The company use 2960 testing hours
sales_basic_used  = A[2,0]*x1 + A[2,1]*x2 # The company sells 560 Basic PC
sales_xp_used  = A[3,0]*x1 + A[3,1]*x2 # The company sells 1200 Basic PC
print(f"Assembly hours used: {assembly_used} / {b[0]}")
print(f"Testing  hours used: {testing_used} / {b[1]}")
print(f"Sales limit (Basic) used: {sales_basic_used} / {b[2]}")
print(f"Sales limit (XP) used: {sales_xp_used} / {b[3]}")

Assembly hours used: 10000.0 / 10000.0
Testing  hours used: 2960.0 / 3000.0
Sales limit (Basic) used: 560.0 / 600.0
Sales limit (XP) used: 1200.0 / 1200.0


**The results above shows that, the constraints on assembling time and XP PC sales are binding, while the constraints on testing time and basic PC sales are not binding.**

## Sensitivity analysis on decision variable


In the following calculation, we will identify two key pieces of information that help us understand the robustness of the optimal solution

### Shadow prices

A shadow price tells us how valuable a resource is at the margin.

- If a constraint is binding (meaning the resource is fully used), its shadow price shows how much the total profit would increase if we had one additional unit of that resource.

- If a constraint is not binding (some of the resource is unused), its shadow price is zero, meaning that increasing this resource would not improve profit.

In simple terms, shadow prices identify which resources are bottlenecks and how costly those bottlenecks are.


### Reduced costs

A reduced cost tells us whether producing more of a product would be profitable under the current conditions.

- If a product is already produced in positive quantity, its reduced cost is zero. This means the product is already profitable given the current resource constraints.

- If a product is not produced, the reduced cost indicates how much its unit profit would need to increase before it would become worthwhile to produce.

In simple terms, reduced costs explain why some products are produced and others are not, given the current profits and resource limits.

In [ ]:
from pyomo.environ import Suffix, Constraint, Var, value, SolverFactory
# 1) Attach suffixes BEFORE solving
model.dual = Suffix(direction=Suffix.IMPORT)  # shadow prices
model.rc   = Suffix(direction=Suffix.IMPORT)  # reduced costs

# 2) Solve (LP)
solver = SolverFactory("appsi_highs")
results = solver.solve(model, tee=False)

# -----------------------
# Shadow prices (duals)
# -----------------------
print("\n=== Shadow Prices (Duals) ===")
for con in model.component_objects(Constraint, active=True):
    for idx in con:
        c = con[idx]
        d = model.dual.get(c, None)
        if d is not None:
            print(f"{con.name}{'' if idx is None else idx}: dual = {d:.6g}")

# -----------------------
# Reduced costs (vars)
# -----------------------
print("\n=== Reduced Costs ===")
for var in model.component_objects(Var, active=True):
    for idx in var:
        v = var[idx]
        rc = model.rc.get(v, None)
        if rc is not None:
            print(f"{var.name}{'' if idx is None else idx}: value = {value(v):.6g}, rc = {rc:.6g}")



=== Shadow Prices (Duals) ===
cons1: dual = 16
cons2: dual = 0
cons3: dual = 0
cons4: dual = 33
cons5: dual = 0
cons6: dual = 0

=== Reduced Costs ===
x1: value = 560, rc = -0
x2: value = 1200, rc = -0


**Result analysis:**

**Shadow Price**
- Shadow price = 16 for assembly (cons1).

If PC Tech could get 1 more assembly hour, profit would increase by about $16 (in the LP sensitivity sense), as long as the “structure” of the solution doesn't change.

- Shadow price = 33 for the XP sales limit (cons4).

If PC Tech could sell 1 more XP computer (i.e., increase the limit from 1200 to 1201), profit would increase by about $33, again for small changes.

The other constraints have shadow price 0. These constraints can not restrict the optimal solution

**Reduced cost**

reduced cost = 0 means you are already producing each product at a level that is consistent with profit maximization under the constraints.

### Allowable increase/decrease on input parameters

The allowable increase/decrease describes how far an input can be changed before the structure of the optimal solution switches. Here, “structure” means the same set of binding constraints(bottlenecks) continues to determine the optimum solution. The bottlenecks remain the same and the shadow prices/reduced costs remain valid.


In [ ]:
c_coef = df.iloc[1, 2:4].astype(float).to_numpy()
A_coef = df.iloc[3:9, 2:4].astype(float).to_numpy()
b_coef = df.iloc[3:9, 5].astype(float).to_numpy()

# --- Use the first 4 constraints for the standard sensitivity table
A4 = A_coef[:4, :]
b4 = b_coef[:4]
m4, n = A4.shape

var_names = ["x1 (Basic)", "x2 (XP)"]
con_names = ["Assembly hours", "Testing hours", "Sales Basic", "Sales XP"]

TOL = 1e-7
INF_DISPLAY = 1e30

solver = pyo.SolverFactory("appsi_highs")

# ============================================================
# 1) Baseline LP
# ============================================================
base = pyo.ConcreteModel(name="LP_base")
base.x1 = pyo.Var(domain=pyo.NonNegativeReals)
base.x2 = pyo.Var(domain=pyo.NonNegativeReals)
base.obj = pyo.Objective(expr=c_coef[0]*base.x1 + c_coef[1]*base.x2, sense=pyo.maximize)

base.cons = pyo.ConstraintList()
for i in range(m4):
    base.cons.add(A4[i,0]*base.x1 + A4[i,1]*base.x2 <= b4[i])

res0 = solver.solve(base)
term0 = str(res0.solver.termination_condition).lower()
if "optimal" not in term0:
    raise RuntimeError(f"Baseline LP not optimal: {res0.solver.termination_condition}")

x0 = np.array([pyo.value(base.x1), pyo.value(base.x2)], dtype=float)
used0 = A4 @ x0
slack0 = b4 - used0
bind0 = tuple((np.abs(slack0) <= TOL).tolist())  # True = binding

# ============================================================
# 2) Allowable ranges for RHS (constraints)
# ============================================================
rhs_allow_inc = np.zeros(m4)
rhs_allow_dec = np.zeros(m4)

for k in range(m4):
    # If nonbinding: allowable increase is infinite; allowable decrease is current slack
    if not bind0[k]:
        rhs_allow_inc[k] = np.inf
        rhs_allow_dec[k] = float(slack0[k])
        continue

    # ---- Allowable increase for b_k ----
    low = 0.0
    high = 1.0

    while True:
        b_test = b4.copy()
        b_test[k] = b_test[k] + high

        tmp = pyo.ConcreteModel()
        tmp.x1 = pyo.Var(domain=pyo.NonNegativeReals)
        tmp.x2 = pyo.Var(domain=pyo.NonNegativeReals)
        tmp.obj = pyo.Objective(expr=c_coef[0]*tmp.x1 + c_coef[1]*tmp.x2, sense=pyo.maximize)

        tmp.cons = pyo.ConstraintList()
        for i in range(m4):
            tmp.cons.add(A4[i,0]*tmp.x1 + A4[i,1]*tmp.x2 <= b_test[i])

        r = solver.solve(tmp)
        term = str(r.solver.termination_condition).lower()
        if "optimal" not in term:
            break

        x_test = np.array([pyo.value(tmp.x1), pyo.value(tmp.x2)], dtype=float)
        used_test = A4 @ x_test
        slack_test = b_test - used_test
        bind_test = tuple((np.abs(slack_test) <= TOL).tolist())

        if bind_test == bind0:
            low = high
            high *= 2.0
            if high > 1e6:
                high = np.inf
                break
        else:
            break

    if np.isinf(high):
        rhs_allow_inc[k] = np.inf
    else:
        for _ in range(60):
            mid = 0.5*(low + high)
            b_test = b4.copy()
            b_test[k] = b_test[k] + mid

            tmp = pyo.ConcreteModel()
            tmp.x1 = pyo.Var(domain=pyo.NonNegativeReals)
            tmp.x2 = pyo.Var(domain=pyo.NonNegativeReals)
            tmp.obj = pyo.Objective(expr=c_coef[0]*tmp.x1 + c_coef[1]*tmp.x2, sense=pyo.maximize)

            tmp.cons = pyo.ConstraintList()
            for i in range(m4):
                tmp.cons.add(A4[i,0]*tmp.x1 + A4[i,1]*tmp.x2 <= b_test[i])

            r = solver.solve(tmp)
            term = str(r.solver.termination_condition).lower()
            if "optimal" not in term:
                high = mid
                continue

            x_test = np.array([pyo.value(tmp.x1), pyo.value(tmp.x2)], dtype=float)
            used_test = A4 @ x_test
            slack_test = b_test - used_test
            bind_test = tuple((np.abs(slack_test) <= TOL).tolist())

            if bind_test == bind0:
                low = mid
            else:
                high = mid

        rhs_allow_inc[k] = low

    # ---- Allowable decrease for b_k ----
    low = 0.0
    high = 1.0

    while True:
        b_test = b4.copy()
        b_test[k] = b_test[k] - high

        tmp = pyo.ConcreteModel()
        tmp.x1 = pyo.Var(domain=pyo.NonNegativeReals)
        tmp.x2 = pyo.Var(domain=pyo.NonNegativeReals)
        tmp.obj = pyo.Objective(expr=c_coef[0]*tmp.x1 + c_coef[1]*tmp.x2, sense=pyo.maximize)

        tmp.cons = pyo.ConstraintList()
        for i in range(m4):
            tmp.cons.add(A4[i,0]*tmp.x1 + A4[i,1]*tmp.x2 <= b_test[i])

        r = solver.solve(tmp)
        term = str(r.solver.termination_condition).lower()
        if "optimal" not in term:
            break

        x_test = np.array([pyo.value(tmp.x1), pyo.value(tmp.x2)], dtype=float)
        used_test = A4 @ x_test
        slack_test = b_test - used_test
        bind_test = tuple((np.abs(slack_test) <= TOL).tolist())

        if bind_test == bind0:
            low = high
            high *= 2.0
            if high > 1e6:
                high = np.inf
                break
        else:
            break

    if np.isinf(high):
        rhs_allow_dec[k] = np.inf
    else:
        for _ in range(60):
            mid = 0.5*(low + high)
            b_test = b4.copy()
            b_test[k] = b_test[k] - mid

            tmp = pyo.ConcreteModel()
            tmp.x1 = pyo.Var(domain=pyo.NonNegativeReals)
            tmp.x2 = pyo.Var(domain=pyo.NonNegativeReals)
            tmp.obj = pyo.Objective(expr=c_coef[0]*tmp.x1 + c_coef[1]*tmp.x2, sense=pyo.maximize)

            tmp.cons = pyo.ConstraintList()
            for i in range(m4):
                tmp.cons.add(A4[i,0]*tmp.x1 + A4[i,1]*tmp.x2 <= b_test[i])

            r = solver.solve(tmp)
            term = str(r.solver.termination_condition).lower()
            if "optimal" not in term:
                high = mid
                continue

            x_test = np.array([pyo.value(tmp.x1), pyo.value(tmp.x2)], dtype=float)
            used_test = A4 @ x_test
            slack_test = b_test - used_test
            bind_test = tuple((np.abs(slack_test) <= TOL).tolist())

            if bind_test == bind0:
                low = mid
            else:
                high = mid

        rhs_allow_dec[k] = low

# ============================================================
# 3) Allowable ranges for objective coefficients (decision variables)
# ============================================================
c_allow_inc = np.zeros(n)
c_allow_dec = np.zeros(n)

for j in range(n):
    # ---- Allowable increase for c_j ----
    low = 0.0
    high = 1.0

    while True:
        c_test = c_coef.copy()
        c_test[j] = c_test[j] + high

        tmp = pyo.ConcreteModel()
        tmp.x1 = pyo.Var(domain=pyo.NonNegativeReals)
        tmp.x2 = pyo.Var(domain=pyo.NonNegativeReals)
        tmp.obj = pyo.Objective(expr=c_test[0]*tmp.x1 + c_test[1]*tmp.x2, sense=pyo.maximize)

        tmp.cons = pyo.ConstraintList()
        for i in range(m4):
            tmp.cons.add(A4[i,0]*tmp.x1 + A4[i,1]*tmp.x2 <= b4[i])

        r = solver.solve(tmp)
        term = str(r.solver.termination_condition).lower()
        if "optimal" not in term:
            break

        x_test = np.array([pyo.value(tmp.x1), pyo.value(tmp.x2)], dtype=float)
        used_test = A4 @ x_test
        slack_test = b4 - used_test
        bind_test = tuple((np.abs(slack_test) <= TOL).tolist())

        if bind_test == bind0:
            low = high
            high *= 2.0
            if high > 1e6:
                high = np.inf
                break
        else:
            break

    if np.isinf(high):
        c_allow_inc[j] = np.inf
    else:
        for _ in range(60):
            mid = 0.5*(low + high)
            c_test = c_coef.copy()
            c_test[j] = c_test[j] + mid

            tmp = pyo.ConcreteModel()
            tmp.x1 = pyo.Var(domain=pyo.NonNegativeReals)
            tmp.x2 = pyo.Var(domain=pyo.NonNegativeReals)
            tmp.obj = pyo.Objective(expr=c_test[0]*tmp.x1 + c_test[1]*tmp.x2, sense=pyo.maximize)

            tmp.cons = pyo.ConstraintList()
            for i in range(m4):
                tmp.cons.add(A4[i,0]*tmp.x1 + A4[i,1]*tmp.x2 <= b4[i])

            r = solver.solve(tmp)
            term = str(r.solver.termination_condition).lower()
            if "optimal" not in term:
                high = mid
                continue

            x_test = np.array([pyo.value(tmp.x1), pyo.value(tmp.x2)], dtype=float)
            used_test = A4 @ x_test
            slack_test = b4 - used_test
            bind_test = tuple((np.abs(slack_test) <= TOL).tolist())

            if bind_test == bind0:
                low = mid
            else:
                high = mid

        c_allow_inc[j] = low

    # ---- Allowable decrease for c_j ----
    low = 0.0
    high = 1.0

    while True:
        c_test = c_coef.copy()
        c_test[j] = c_test[j] - high

        tmp = pyo.ConcreteModel()
        tmp.x1 = pyo.Var(domain=pyo.NonNegativeReals)
        tmp.x2 = pyo.Var(domain=pyo.NonNegativeReals)
        tmp.obj = pyo.Objective(expr=c_test[0]*tmp.x1 + c_test[1]*tmp.x2, sense=pyo.maximize)

        tmp.cons = pyo.ConstraintList()
        for i in range(m4):
            tmp.cons.add(A4[i,0]*tmp.x1 + A4[i,1]*tmp.x2 <= b4[i])

        r = solver.solve(tmp)
        term = str(r.solver.termination_condition).lower()
        if "optimal" not in term:
            break

        x_test = np.array([pyo.value(tmp.x1), pyo.value(tmp.x2)], dtype=float)
        used_test = A4 @ x_test
        slack_test = b4 - used_test
        bind_test = tuple((np.abs(slack_test) <= TOL).tolist())

        if bind_test == bind0:
            low = high
            high *= 2.0
            if high > 1e6:
                high = np.inf
                break
        else:
            break

    if np.isinf(high):
        c_allow_dec[j] = np.inf
    else:
        for _ in range(60):
            mid = 0.5*(low + high)
            c_test = c_coef.copy()
            c_test[j] = c_test[j] - mid

            tmp = pyo.ConcreteModel()
            tmp.x1 = pyo.Var(domain=pyo.NonNegativeReals)
            tmp.x2 = pyo.Var(domain=pyo.NonNegativeReals)
            tmp.obj = pyo.Objective(expr=c_test[0]*tmp.x1 + c_test[1]*tmp.x2, sense=pyo.maximize)

            tmp.cons = pyo.ConstraintList()
            for i in range(m4):
                tmp.cons.add(A4[i,0]*tmp.x1 + A4[i,1]*tmp.x2 <= b4[i])

            r = solver.solve(tmp)
            term = str(r.solver.termination_condition).lower()
            if "optimal" not in term:
                high = mid
                continue

            x_test = np.array([pyo.value(tmp.x1), pyo.value(tmp.x2)], dtype=float)
            used_test = A4 @ x_test
            slack_test = b4 - used_test
            bind_test = tuple((np.abs(slack_test) <= TOL).tolist())

            if bind_test == bind0:
                low = mid
            else:
                high = mid

        c_allow_dec[j] = low

# ============================================================
# 4) Output tables (allowable ranges only)
# ============================================================
var_table = pd.DataFrame({
    "Variable": var_names,
    "Objective Coefficient": [c_coef[0], c_coef[1]],
    "Allowable Increase": [INF_DISPLAY if np.isinf(c_allow_inc[0]) else c_allow_inc[0],
                           INF_DISPLAY if np.isinf(c_allow_inc[1]) else c_allow_inc[1]],
    "Allowable Decrease": [INF_DISPLAY if np.isinf(c_allow_dec[0]) else c_allow_dec[0],
                           INF_DISPLAY if np.isinf(c_allow_dec[1]) else c_allow_dec[1]],
})
display(var_table)

con_table = pd.DataFrame({
    "Constraint": con_names,
    "Constraint R.H. Side": b4,
    "Allowable Increase": [INF_DISPLAY if np.isinf(rhs_allow_inc[i]) else rhs_allow_inc[i] for i in range(m4)],
    "Allowable Decrease": [INF_DISPLAY if np.isinf(rhs_allow_dec[i]) else rhs_allow_dec[i] for i in range(m4)],
})
display(con_table)

,Variable,Objective Coefficient,Allowable Increase,Allowable Decrease
0,x1 (Basic),80.0,2.750000e+01,80.0
1,x2 (XP),129.0,1.000000e+30,33.0


,Constraint,Constraint R.H. Side,Allowable Increase,Allowable Decrease
0,Assembly hours,10000.0,2.000000e+02,2800.000001
1,Testing hours,3000.0,1.000000e+30,40.000000
2,Sales Basic,600.0,1.000000e+30,40.000000
3,Sales XP,1200.0,5.000000e+01,33.333333
